# Course project: cleavage plane for Ca doped YBCO

In [27]:
import numpy as np, os
import matplotlib.pyplot as plt
from pwscf import write_pwscf_input
from labutil.plugins.pwscf import run_qe_pwscf, PWscf_inparam, parse_qe_pwscf_output
from labutil.objects import Struc, Dir, ase2struc, Kpoints, PseudoPotential
from ase.spacegroup import crystal
from ase.io import write
from ase.build import bulk, make_supercell, add_vacuum
from ase import Atoms
import numpy, os, copy
from labutil.objects import TextFile, ExternalCode, File, Param
from labutil.util import prepare_dir, run_command

Make a structure from standard cell lattice parameters.

In [46]:
def make_struc_non_doped(nxy=1, nz = 2, alat=3.82, blat=3.89, clat=11.68, vacuum=0, cleave_plane='NO',
                         separation=0):
    """
    Creates the crystal structure using ASE.
    :param alat: Lattice parameter in angstrom
    :return: structure object converted from ase
    """
    a = np.sqrt(alat**2 + blat**2)
    lattice = np.array([[a,0,0],[0,a,0],[0,0,clat]])
    symbols = ['Y', 'Y', 'Ba', 'Ba', 'Ba', 'Ba', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu','O', 'O', 'O','O','O','O','O','O','O','O','O','O','O','O',]
    sc_pos = [[0.5,0,0.5], [0,0.5,0.5], 
              [0.5, 0, 0.81939], [0, 0.5, 0.81939], [0.5, 0, 0.18061], [0, 0.5, 0.18061],
              [0,0,0], [0.5,0.5,0], [0,0,0.64668], [0.5,0.5,0.64668], [0,0,0.35332], [0.5,0.5,0.35332],
              [0.25,0.25,0], [0.75,0.75,0], [0,0,0.15918], [0.5,0.5,0.15918], [0.25,0.25,0.37835], [0.75,0.75,0.37835],[0.25,0.75,0.37935], [0.75,0.25,0.37935], [0.25,0.25,0.62065], [0.75,0.75,0.62065],[0.25,0.75,0.62165], [0.75,0.25,0.62165], [0,0,0.84082], [0.5,0.5,0.84082]
             ]
    YBCO = Atoms(symbols=symbols, scaled_positions=sc_pos, cell=lattice)
    multiplier = np.identity(3)
    multiplier[0,0]=nxy
    multiplier[1,1]=nxy
    multiplier[2,2]=nz
    supercell = make_supercell(YBCO, multiplier)
    add_vacuum(supercell, vacuum)
    write(f'YBCO_{nxy}{nxy}{nz}_{vacuum}vac_{cleave_plane}cleave_{separation}sep.cif', supercell)
    structure = Struc(ase2struc(supercell))
    return structure

In [49]:
struc = make_struc_non_doped(nxy = 1, nz = 3, vacuum = 40)

Struc {'cell': [[5.452017975025394, 0.0, 0.0], [0.0, 5.452017975025394, 0.0], [0.0, 0.0, 35.04]], 'positions': [['Y', [2.726008987512697, 0.0, 5.84]], ['Y', [0.0, 2.726008987512697, 5.84]], ['Ba', [2.726008987512697, 0.0, 9.570475199999999]], ['Ba', [0.0, 2.726008987512697, 9.570475199999999]], ['Ba', [2.726008987512697, 0.0, 2.1095248]], ['Ba', [0.0, 2.726008987512697, 2.1095248]], ['Cu', [0.0, 0.0, 0.0]], ['Cu', [2.726008987512697, 2.726008987512697, 0.0]], ['Cu', [0.0, 0.0, 7.5532224]], ['Cu', [2.726008987512697, 2.726008987512697, 7.5532224]], ['Cu', [0.0, 0.0, 4.1267776000000005]], ['Cu', [2.726008987512697, 2.726008987512697, 4.1267776000000005]], ['O', [1.3630044937563486, 1.3630044937563486, 0.0]], ['O', [4.0890134812690455, 4.0890134812690455, 0.0]], ['O', [0.0, 0.0, 1.8592223999999997]], ['O', [2.726008987512697, 2.726008987512697, 1.8592223999999997]], ['O', [1.3630044937563486, 1.3630044937563486, 4.419128]], ['O', [4.0890134812690455, 4.0890134812690455, 4.419128]], ['O', 

Inital relax calculation (RUNS A CALC)

In [ ]:
def relax_YBCO(nk, ecut):
    pseudopots = {'Y': PseudoPotential(ptype='uspp', element='Y', functional='LDA', name='Y.pz-spn-rrkjus_psl.1.0.0.UPF'),
                  'Ba': PseudoPotential(ptype='uspp', element='Ba', functional='LDA', name='Ba.pz-spn-rrkjus_psl.1.0.0.UPF'),
                  'Cu': PseudoPotential(ptype='uspp', element='Cu', functional='LDA', name='Cu.pz-d-rrkjus.UPF'),
                  'O': PseudoPotential(ptype='uspp', element='O', functional='LDA', name='O.pz-rrkjus.UPF')}
    struc = make_struc_non_doped()
    kpts = Kpoints(gridsize=[nk, nk, nk], option='automatic', offset=True)
    dirname = 'YBCO_relax_ecut_{}_nk_{}'.format(ecut, nk)
    runpath = Dir(path=os.path.join(os.environ['WORKDIR'], "Project", dirname))
    input_params = PWscf_inparam({
        'CONTROL': {
            'calculation': 'relax',
            'pseudo_dir': os.environ['QE_POTENTIALS'],
            'outdir': runpath.path,
            'tstress': True,
            'tprnfor': True,
            'disk_io': 'none'
        },
        'SYSTEM': {
            'ecutwfc': ecut,
            'ecutrho': ecut * 12,
            'occupations': 'smearing',
            'smearing': 'mp',
            'degauss': 0.02
             },
        'ELECTRONS': {
            'diagonalization': 'david',
            'mixing_beta': 0.7,
            'conv_thr': 1e-7,
        },
        'IONS': {
            'ion_dynamics': 'bfgs'
        },
        'CELL': {},
        })

    output_file = run_qe_pwscf(runpath=runpath, struc=struc,  pseudopots=pseudopots,
                               params=input_params, kpoints=kpts, ncpu=16)
    output = parse_qe_pwscf_output(outfile=output_file)
    return output

def kpts_scan():
    ecut = 30
    nk = np.arange(1, 2)
    energy=[]
    for i in nk:
        output = relax_YBCO(ecut=ecut, nk=i)
        energy.append(output['energy'])
        print(i)
    return energy